<a href="https://colab.research.google.com/github/AUT-Student/NLP-HW3/blob/main/NLP_HW3_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
!pip install bert_embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.8 MB 30.1 MB/s 
     |████████████████████████████████| 209 kB 59.0 MB/s 
     |████████████████████████████████| 29.6 MB 1.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.6.0-py3-none-any.whl size=259930 sha256=cc2aa0e670ffcb8f104a5c71b705b4f1fdcf8d0185628e3c260551d970934781
  Stored in directory: /root/.cache/pip/wheels/a6/41/8f/45bd1c58055d87aee5a71b6756a427ea8d92e506b3a9d17370
Successfully built gluonnlp
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [31]:
import pandas as pd
import numpy as np

from bert_embedding import BertEmbedding
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score

# Dataset 

In [3]:
!gdown 1oin_Sw1Gk_WLS9zpDrap5FdlRcUrTP_D
!gdown 1EvtGQ8-sYXXQ3VA9ByjD4OmFH13WxvAs
!gdown 154f-z0PsPAp0yvOLdNXP8NXYjPljpgCZ
!gdown 1EDG_j6F5ohIjpkihRQd0-i9daHWV7StN

Downloading...
From: https://drive.google.com/uc?id=1oin_Sw1Gk_WLS9zpDrap5FdlRcUrTP_D
To: /content/Sentenses_train.txt
100% 1.23M/1.23M [00:00<00:00, 77.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EvtGQ8-sYXXQ3VA9ByjD4OmFH13WxvAs
To: /content/Senses_train.txt
100% 49.7k/49.7k [00:00<00:00, 57.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=154f-z0PsPAp0yvOLdNXP8NXYjPljpgCZ
To: /content/Sentenses_test.txt
100% 213k/213k [00:00<00:00, 86.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EDG_j6F5ohIjpkihRQd0-i9daHWV7StN
To: /content/Senses_test.txt
100% 8.76k/8.76k [00:00<00:00, 13.6MB/s]


In [4]:
def create_dataset(part_name):
  sentences_file = open(f"/content/Sentenses_{part_name}.txt", "r")
  senses_file = open(f"/content/Senses_{part_name}.txt", "r")

  sentences = sentences_file.readlines()
  senses = senses_file.readlines()
  
  dataset_sentence = []
  dataset_sense = []
  dataset_ambiguity_word = []

  for sentence, sense in list(zip(sentences, senses)):
    dataset_sense.append(sense[:-1])
    tokens = sentence.split()
    word_index = tokens.index("<head>")
    dataset_ambiguity_word.append(tokens[word_index+1].lower())
    dataset_sentence.append(" ".join(
        tokens[max(0, word_index-11):word_index-1] +\
        [tokens[word_index+1]] +\
        tokens[word_index+3:min(word_index+13, len(tokens))])
    )
  
  return pd.DataFrame({"Sentence": dataset_sentence, "Ambiguity_Word": dataset_ambiguity_word, "Sense": dataset_sense})

In [5]:
train_dataset = create_dataset("train")
test_dataset = create_dataset("test")

In [14]:
train_dataset = train_dataset[:1200]

# Embedding

In [15]:
# GPU
# ctx = mx.gpu(0)
# bert_embedding = BertEmbedding(ctx=ctx)

# CPU
bert_embedding = BertEmbedding()

Vocab file is not found. Downloading.


In [16]:
def calculate_ambiguity_vector(dataset):
  dataset["Ambiguity_Vector"] = None
  for i, data in dataset.iterrows():
    if i%300==0: print(f"{int(i*100/len(dataset))}%")
    tokens, vectors = bert_embedding([data["Sentence"]])[0]
    ambiguity_word = data["Ambiguity_Word"]

    try:    
      ambiguity_vector = vectors[tokens.index(ambiguity_word)]
    except ValueError:
      tokens, vectors = bert_embedding([data["Sentence"][15:]])[0]
      ambiguity_word = data["Ambiguity_Word"]
      ambiguity_vector = vectors[tokens.index(ambiguity_word)]

    dataset.at[i, "Ambiguity_Vector"] = ambiguity_vector

In [17]:
calculate_ambiguity_vector(train_dataset)

0%
25%
50%
75%


In [18]:
calculate_ambiguity_vector(test_dataset)

0%
26%
53%
80%


# Dimension Reduction

In [19]:
pca = PCA(n_components=300)

In [20]:
pca.fit(np.array([item for item in train_dataset["Ambiguity_Vector"].values]))

PCA(n_components=300)

In [21]:
pca.transform(np.array([item for item in train_dataset["Ambiguity_Vector"].values])).shape

(1200, 300)

In [22]:
train_dataset["Ambiguity_Vector_Reduced"] = [item for item in pca.transform(np.array([item for item in train_dataset["Ambiguity_Vector"].values]))]

In [23]:
test_dataset["Ambiguity_Vector_Reduced"] = [item for item in pca.transform(np.array([item for item in test_dataset["Ambiguity_Vector"].values]))]

# Train Classifiers

In [7]:
set(train_dataset["Sense"].values)

{'HARD1',
 'HARD2',
 'HARD3',
 'SERVE10',
 'SERVE12',
 'cord',
 'division',
 'formation',
 'interest1',
 'interest2',
 'interest3',
 'interest4',
 'interest5',
 'interest6',
 'phone',
 'product',
 'text'}

In [8]:
set(train_dataset["Ambiguity_Word"].values)

{'hard',
 'harder',
 'hardest',
 'interest',
 'interested',
 'interesting',
 'interests',
 'line',
 'lines',
 'serve',
 'served',
 'serves'}

In [9]:
hard_words = ["hard", "harder", "hardest"]
interest_words = ["interest", "interested", "interesting", "interests"]
line_words = ["line", "lines"]
serve_words = ["serve", "served", "serves"]

hard_senses = ["HARD1", "HARD2", "HARD3"]
interest_senses = ["interest1", "interest2", "interest3", "interest4", "interest5", "interest6"]
line_senses = ["cord", "division", "formation", "phone", "product", "text"]
serve_senses = ["SERVE10", "SERVE12"]

mapping_word_sense = {
    "HARD": (hard_words, hard_senses),
    "INTEREST": (interest_words, interest_senses),
    "LINE": (line_words, line_senses),
    "SERVE": (serve_words, serve_senses)
}

In [45]:
def report_accuracy_f(classifier, dataset):
  y_predict = classifier.predict(np.stack(dataset["Ambiguity_Vector_Reduced"].values))
  y_true = dataset["Sense"].values

  print(f"Accuracy = {accuracy_score(y_true, y_predict)}")
  print(f"F1 = {f1_score(y_true, y_predict, average='macro')}")  

In [47]:
classifiers = {}

for key in mapping_word_sense.keys():
  print(key)
  words = mapping_word_sense[key][0]
  senses = mapping_word_sense[key][1]

  sub_dataset = train_dataset[train_dataset["Ambiguity_Word"].isin(words) & train_dataset["Sense"].isin(senses)]

  classifier = SVC()

  if key != "HARD":
    classifier.fit(X=np.stack(sub_dataset["Ambiguity_Vector_Reduced"].values), y=sub_dataset["Sense"].values)
    report_accuracy_f(classifier, sub_dataset)
    classifiers[key] = classifier

HARD
INTEREST
Accuracy = 0.9563758389261745
F1 = 0.7550486797767185
LINE
Accuracy = 0.9711191335740073
F1 = 0.96479310543866
SERVE
Accuracy = 1.0
F1 = 1.0


In [29]:
classifiers

{'INTEREST': SVC(), 'LINE': SVC(), 'SERVE': SVC()}

In [56]:
sub_dataset["Sense"].values

array(['interest6', 'interest3', 'interest6', 'interest6', 'interest6',
       'interest3', 'interest6', 'interest6', 'interest6', 'interest5',
       'interest5', 'interest6', 'interest6', 'interest1', 'interest6',
       'interest1', 'interest6', 'interest6', 'interest6', 'interest6',
       'interest4', 'interest6', 'interest6', 'interest6', 'interest6',
       'interest6', 'interest1', 'interest5', 'interest6', 'interest6',
       'interest6', 'interest1', 'interest6', 'interest1', 'interest6',
       'interest5', 'interest6', 'interest6', 'interest6', 'interest6',
       'interest6', 'interest6', 'interest5', 'interest6', 'interest1',
       'interest6', 'interest1', 'interest6', 'interest5', 'interest6',
       'interest6', 'interest6', 'interest6', 'interest1', 'interest5',
       'interest6', 'interest5', 'interest6', 'interest1', 'interest5',
       'interest1', 'interest1', 'interest6', 'interest6', 'interest1',
       'interest5', 'interest1', 'interest1', 'interest6', 'inte

In [55]:
svc.predict(np.stack(sub_dataset["Ambiguity_Vector_Reduced"].values))

array(['interest6', 'interest1', 'interest6', 'interest6', 'interest6',
       'interest3', 'interest6', 'interest6', 'interest6', 'interest5',
       'interest5', 'interest6', 'interest6', 'interest1', 'interest6',
       'interest1', 'interest6', 'interest6', 'interest6', 'interest6',
       'interest4', 'interest6', 'interest6', 'interest6', 'interest6',
       'interest6', 'interest5', 'interest6', 'interest6', 'interest6',
       'interest6', 'interest1', 'interest6', 'interest1', 'interest6',
       'interest6', 'interest6', 'interest6', 'interest6', 'interest6',
       'interest6', 'interest6', 'interest5', 'interest6', 'interest1',
       'interest6', 'interest1', 'interest6', 'interest5', 'interest6',
       'interest6', 'interest6', 'interest6', 'interest1', 'interest5',
       'interest6', 'interest6', 'interest6', 'interest1', 'interest5',
       'interest1', 'interest1', 'interest6', 'interest6', 'interest1',
       'interest5', 'interest1', 'interest1', 'interest6', 'inte